In [1]:
!pip install tensorflow numpy


In [28]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import collections
import random

print("TensorFlow Version:", tf.__version__)

TensorFlow Version: 2.18.0


In [29]:
text = """
The average person has about 70,000 thoughts a day.Your nose can remember up to 50,000 different scents.The brain has no pain receptors — it can't feel pain itself
The first AI program was written in 1951 to play checkers.GPT-4 can generate, summarize, and translate text — and even write code.AI can now detect some diseases more accurately than human doctors.
Bamboo can grow up to 3 feet in just one day.Earth is the only known planet with plate tectonics.
Deep learning models require vast amounts of data.
"""

# Convert text to lowercase for consistency
text = text.lower()

# Create a sorted list of unique characters in the text
vocab = sorted(list(set(text)))
char_to_int = {char: i for i, char in enumerate(vocab)}
int_to_char = {i: char for i, char in enumerate(vocab)}

vocab_size = len(vocab)
print(f"Vocabulary size: {vocab_size}")
print(f"Vocabulary: {vocab}")

# Prepare training sequences
seq_length = 50 # Length of input sequences
data_X = [] # Input sequences
data_y = [] # Output characters (next character in sequence)

# Create sequences of characters and their corresponding next character
for i in range(0, len(text) - seq_length):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    data_X.append([char_to_int[char] for char in seq_in])
    data_y.append(char_to_int[seq_out])

n_patterns = len(data_X)
print(f"Total patterns: {n_patterns}")

# Reshape X to be [samples, time steps] for Embedding layer input
X = np.reshape(data_X, (n_patterns, seq_length))

# One-hot encode the output variable (y) - not needed for sparse_categorical_crossentropy
y = np.array(data_y)

Vocabulary size: 40
Vocabulary: ['\n', ' ', "'", ',', '-', '.', '0', '1', '3', '4', '5', '7', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—']
Total patterns: 462


In [30]:
model = Sequential([
    # Embedding layer maps integer indices to dense vectors
    Embedding(input_dim=vocab_size, output_dim=256, input_length=seq_length, input_shape=(seq_length,)),
    # LSTM layer to learn sequential dependencies
    LSTM(256, return_sequences=True), # return_sequences=True for stacking LSTMs
    Dropout(0.2), # Dropout for regularization
    LSTM(256), # Last LSTM layer does not return sequences
    Dropout(0.2),
    # Dense layer for output, with softmax activation for probability distribution over vocabulary
    Dense(vocab_size, activation='softmax')
])

# Use Adam optimizer
optimizer = Adam(learning_rate=0.005)
# Use sparse_categorical_crossentropy because y is integer-encoded, not one-hot
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer)

model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 50, 256)        │        10,240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_5 (LSTM)                   │ (None, 50, 256)        │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 50, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 256)            │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 40)             │        10,280 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,071,144 (4.09 MB)

 Trainable params: 1,071,144 (4.09 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:

print("\nStarting model training...")
# Train the model
# Using a small number of epochs for demonstration.
# For better results, increase epochs and use callbacks like ModelCheckpoint.
history = model.fit(X, y, epochs=20, batch_size=128, verbose=1)
print("Model training complete.")


Starting model training...
Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - loss: 3.7036
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 829ms/step - loss: 3.3022
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 854ms/step - loss: 3.1430
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 1s/step - loss: 4.0784
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 844ms/step - loss: 3.0672
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 826ms/step - loss: 3.1069
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 3.0122
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 841ms/step - loss: 2.9944
Epoch 9/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 853ms/step - loss: 3.0313
Epoch 10/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 995ms/step - loss: 2.9494
Epoch 11/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 862ms/step - loss: 2.8948
Epoch 12/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 1s/step - loss: 2.8398
Epoch 13/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 837ms/step - loss: 2.8013
Epoch 14/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 5s 856ms/step - loss: 2.7452
Epoch 15/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 943ms/step - loss: 2.7223
Epo

In [32]:
def generate_text(model, seed_text, num_chars_to_generate=200, temperature=1.0):
    """
    Generates text using the trained LSTM model.

    Args:
        model (tf.keras.Model): The trained Keras model.
        seed_text (str): The initial text to start generation from.
        num_chars_to_generate (int): The number of characters to generate.
        temperature (float): Controls the randomness of predictions.
                             Lower temp (e.g., 0.2) makes text more predictable.
                             Higher temp (e.g., 1.0) makes text more random/creative.
    Returns:
        str: The generated text.
    """
    generated_text = seed_text.lower()
    pattern = [char_to_int[char] for char in generated_text[-seq_length:]] # Use last part of seed if too long

    print(f"\nGenerating text with seed: \"{seed_text}\"")
    print(f"Temperature: {temperature}")

    for i in range(num_chars_to_generate):
        # Reshape input for the model: (1, seq_length, 1)
        x = np.reshape(pattern, (1, len(pattern), 1))
        # Predict probabilities for the next character
        prediction_probabilities = model.predict(x, verbose=0)[0]

        # Apply temperature to probabilities
        prediction_probabilities = np.log(prediction_probabilities) / temperature
        exp_preds = np.exp(prediction_probabilities)
        prediction_probabilities = exp_preds / np.sum(exp_preds)

        # Sample the next character based on probabilities
        next_char_int = np.random.choice(len(vocab), p=prediction_probabilities)
        next_char = int_to_char[next_char_int]

        generated_text += next_char
        # Update the pattern for the next prediction
        pattern.append(next_char_int)
        pattern = pattern[1:len(pattern)] # Keep the pattern length consistent

    return generated_text

In [33]:
print("\n" + "="*50)
print("Demonstrating Text Generation")
print("="*50 + "\n")

# Example 1: Generate text with a common seed
seed1 = "The average person has about 70,000 thoughts"
generated_output1 = generate_text(model, seed1, num_chars_to_generate=150, temperature=0.5)
print(f"\nGenerated Text 1:\n{generated_output1}")
print("\n" + "="*50 + "\n")

# Example 2: Generate text with a different seed and higher temperature
seed2 = "The first AI program was written in 1951"
generated_output2 = generate_text(model, seed2, num_chars_to_generate=150, temperature=1.0)
print(f"\nGenerated Text 2:\n{generated_output2}")
print("\n" + "="*50 + "\n")

# Example 3: Generate text with a very short seed
seed3 = "Bamboo can grow"
generated_output3 = generate_text(model, seed3, num_chars_to_generate=150, temperature=0.6)
print(f"\nGenerated Text 3:\n{generated_output3}")
print("\n" + "="*50 + "\n")
# Example 4: Generate text with a very short seed
seed4 = "deep learning"
generated_output4 = generate_text(model, seed4, num_chars_to_generate=150, temperature=0.7)
print(f"\nGenerated Text 4:\n{generated_output3}")
print("\n" + "="*50 + "\n")


Demonstrating Text Generation


Generating text with seed: "The average person has about 70,000 thoughts"
Temperature: 0.5

Generated Text 1:
the average person has about 70,000 thoughtses asette yot an tate daan day nen in.eiin dopin yan cane.
pare can canes tote ane mate gan ain knogreren — paecte dereen — wpan cere pate wn ppan tan



Generating text with seed: "The first AI program was written in 1951"
Temperature: 1.0

Generated Text 2:
the first ai program was written in 1951l looo fefese 5ait dtate kiae.pamee as0lmuor0e id.—os — dafi,.wegsn tin ohbarbf a— 
tte cin.amamecs manorin citente— lhan wqcmans c maen 5ouura pon se



Generating text with seed: "Bamboo can grow"
Temperature: 0.6

Generated Text 3:
bamboo can growas cen ait yor pmael detertte tete toin — n inet praise phe man.dete ron tate san ranitt no pan tote can ratel.mar ane an fate0 nepsemen cheres mamait



Generating text with seed: "deep learning"
Temperature: 0.7

Generated Text 4:
bamboo can growas cen ait yor 